Требования к нереляционным БД \
2 - нереляционная БД \
2 - красивая структура БД \
2 - интерфейс позволяет класть, доставать, удалять данные (проводить операции CRUD - Create, Read, Update, Delete) \
2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...) \
2 - Neo4j - нахождение путей не только к соседним вершинам, операции с графами 


In [327]:
import wptools
import re
import datetime
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver("bolt://174.129.45.71:7687", auth=basic_auth("neo4j", "pencils-ringing-screw"))
session = driver.session()

Добавим в базу Пола Маккартни.

In [328]:
name = 'Paul McCartney'

In [329]:
def get_bands(artist):
    so = wptools.page(artist).get_parse()
    infobox = so.data['infobox']
    try:
        infobox = infobox['module']
        infobox = infobox.replace("'","").replace(':','').replace('[','').replace(']','').replace('*','').replace('nowrap','')
        infobox = infobox.replace("flatlist","").replace('{','').replace('}','').replace('|','').replace('=','')
        
        infobox = infobox.split('\n')
        for el in infobox:
            if 'associated_acts' in el:
                need = el
        index1 = infobox.index(need)
        infobox1 = infobox[index1+1:]
        
        infobox = []
        for k in infobox1:
            if k != ' ':
                k = k.split(')')
                if '(' in k[0]:
                    infobox.append(k[0].strip()+')')
                else:
                    infobox.append(k[0].strip())
            else:
                break
        
    except:
        try:
            infobox = infobox['associated_acts']
        except:
            try: 
                infobox = infobox['Associated_acts']
            except:
                infobox = 'nothing found'
    return infobox

In [330]:
bands = get_bands(name)

en.wikipedia.org (parse) Paul McCartney
en.wikipedia.org (imageinfo) File:Paul McCartney 2021 (cropped).jpg
Paul McCartney (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Paul McCa...
  infobox: <dict(22)> honorific_prefix, name, honorific_suffix, im...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Paul_McCar...
  pageid: 13736167
  parsetree: <str(359670)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Paul McCartney
  wikibase: Q2599
  wikidata_url: https://www.wikidata.org/wiki/Q2599
  wikitext: <str(247278)> {{Short description|English musician, me...
}


In [331]:
bands

['The Quarrymen',
 'The Beatles',
 'Wings (band)',
 'The Fireman (band)',
 'Linda McCartney']

In [332]:
def get_data_place(artist):
    so = wptools.page(artist).get_parse()
    infobox = so.data['infobox']
    try:
        birth_date = infobox['birth_date']
        nums = re.findall(r'\d+', birth_date)
        date = datetime.datetime(int(nums[0]), int(nums[1]), int(nums[2]))
        now = datetime.datetime.now()
        age = round((now-date).days/365.25)
    except: 
        age = 'None'
    try:
        birth_place = infobox['birth_place'].split(',')[-1].strip()
    except:
        birth_place = 'None'
    return birth_place, age

In [333]:
birth_place, age = get_data_place(name)

en.wikipedia.org (parse) Paul McCartney
en.wikipedia.org (imageinfo) File:Paul McCartney 2021 (cropped).jpg
Paul McCartney (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Paul McCa...
  infobox: <dict(22)> honorific_prefix, name, honorific_suffix, im...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Paul_McCar...
  pageid: 13736167
  parsetree: <str(359670)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Paul McCartney
  wikibase: Q2599
  wikidata_url: https://www.wikidata.org/wiki/Q2599
  wikitext: <str(247278)> {{Short description|English musician, me...
}


In [334]:
birth_place, age

('England', 80)

### Create, read

In [335]:
def add_artist(name, bands, birth_place, age):
    add_person = 'CREATE (p:Person {name: "'+name+'", age: "'+str(age)+'"})'
    _ = session.run(add_person)
    
    for band in bands:
        add_band = 'MERGE (g:Group {name: "'+band+'"})' 
        _ = session.run(add_band)
        
        edge_band = 'MATCH (p:Person {name: "'+name+'"}), (g:Group {name: "'+band+'"}) MERGE (p)-[r:ASSOCIATED_WITH]->(g)'
        _ = session.run(edge_band)

    add_country = 'MERGE (c:Country {name: "'+birth_place+'"})' 
    _ = session.run(add_country)
    
    edge_country = 'MATCH (p:Person {name: "'+name+'"}), (c:Country {name: "'+birth_place+'"}) MERGE (p)-[r:WHERE_FROM]->(c)'
    _ = session.run(edge_country)
    
    get_data = 'MATCH (p:Person {name: "'+name+'"})-[r:WHERE_FROM]-(c) RETURN p.name, p.age, c.name' 
    _ = session.run(get_data)
    metadata = [record.data() for record in _][0]
    
    get_data = 'MATCH (p:Person {name: "'+name+'"})-[r:ASSOCIATED_WITH]-(g) RETURN g.name' 
    r = session.run(get_data)
    result = [record.data() for record in r]
    bands = []
    for band in result:
        bands.append(band['g.name'])

    return('Input was: '+ metadata['p.name'] + ', '+ metadata['p.age']  \
           + ', '+ metadata['c.name'] +  ', ' + ', '.join(bands))

In [336]:
add_artist(name, bands, birth_place, age)

'Input was: Paul McCartney, 80, England, Linda McCartney, The Fireman (band), Wings (band), The Beatles, The Quarrymen'

### Update, Delete

In [337]:
def update_age(name, age):
    update_ = """MATCH (p:Person {name: '%s'})
                    SET p.age = '%s'
                    RETURN p.age""" % (name, age)
    result = session.run(update_)
    return result

In [338]:
result = update_age('Damon Albarn', 356)
for el in result:
    print(el.data()['p.age'])

356


In [339]:
def delete_country(name, country):
    delete_ = """MATCH (p:Person {name: '%s'})-[r:WHERE_FROM]->(:Country {name: '%s'})
                    DELETE r""" % (name, country)
    
    match = """MATCH (p:Person {name: '%s'})-[r:WHERE_FROM]->(c) 
        RETURN c.name""" % name
    result = session.run(delete_)
    country = session.run(match)
    return result

In [340]:
res = delete_country('Simon Tong', 'England')

In [341]:
for el in res:
    print(el.data()['c.name'])

### sort, aggregation, group

In [342]:
# список стран в алфавитном порядке
sort = """MATCH (c:Country)
        RETURN c.name
        ORDER BY c.name
        LIMIT 5"""
result = session.run(sort)

In [343]:
for res in result:
    print(res['c.name'])

Colonial Nigeria|British Nigeria
England
France
Nigeria
None


In [344]:
# список исполнителей из Англии
sort = """MATCH (c:Country {name: 'England'})<-[:WHERE_FROM]-(p)
        RETURN p"""
result = session.run(sort)

In [345]:
for res in result:
    print(res[0]['name'])

Paul McCartney
Simon William Lord
Tricky (musician)
Totally Enormous Extinct Dinosaurs
Actress (musician)
Richard Russell (XL Recordings)
Paul Simonon
Dave Rowntree
Alex James (musician)
Jamie Hewlett
Damon Albarn


In [346]:
# исполнитель с самым большим количеством связей
sort = """MATCH (p: Person)-[:ASSOCIATED_WITH]-(g)
        WITH p, count(g) AS amount
        ORDER BY amount DESC
        RETURN p.name
        LIMIT 1"""
result = session.run(sort)


In [347]:
for res in result:
    print(res['p.name'])

Damon Albarn


In [348]:
# топ стран с максимальным количеством исполнителей
sort = """MATCH (c: Country)-[r:WHERE_FROM]-(Person)
        WITH c, count(Person) AS persons
        ORDER BY persons DESC
        LIMIT 5
        RETURN c.name, persons"""
result = session.run(sort)

In [349]:
for res in result:
    print(res['c.name'])

England
None
Colonial Nigeria|British Nigeria
West Germany (now Germany)
U.S.


In [350]:
# значения страны с максимальным количеством исполнителей
sort = """MATCH (c: Country)-[r:WHERE_FROM]-(Person)
WITH c, count(Person) AS persons
ORDER BY persons DESC
LIMIT 1
WITH c.name as target 
MATCH (c: Country {name: target})-[r:WHERE_FROM]-(Person)
RETURN c, Person"""
result = session.run(sort)

In [351]:
for res in result:
    print(res[1]['name'])

Paul McCartney
Simon William Lord
Tricky (musician)
Totally Enormous Extinct Dinosaurs
Actress (musician)
Richard Russell (XL Recordings)
Paul Simonon
Dave Rowntree
Alex James (musician)
Jamie Hewlett
Damon Albarn


In [352]:
search = """MATCH
  (p1:Person {name: 'Damon Albarn'}),
  (p2:Person {name: 'Paul McCartney'}),
  p = shortestPath((p1)-[*..4]-(p2))
    RETURN p"""
result = session.run(search)

In [353]:
for res in result:
    print(res)

<Record p=<Path start=<Node id=105 labels=frozenset({'Person'}) properties={'name': 'Damon Albarn', 'age': '356'}> end=<Node id=30 labels=frozenset({'Person'}) properties={'name': 'Paul McCartney', 'age': '80'}> size=2>>
